In [6]:
library(stringr)


Attaching package: 'stringr'

The following object is masked from 'package:strucchange':

    boundary



In [1]:
# library
library(caret)          # 샘플링
library(nnet)           # 인공신경망
library(party)          # 의사결정나무
library(randomForest)   # 랜덤 포레스트
library(e1071)          # SVM

Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Loading required package: grid
Loading required package: mvtnorm
Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric

Loading required package: sandwich
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: 'randomForest'

The following object is masked from 'package:ggplot2':

    margin



In [10]:
summary(rice)

      년도            월              쌀       
 Min.   :1996   Min.   : 1.00   Min.   :29360  
 1st Qu.:2002   1st Qu.: 3.75   1st Qu.:36021  
 Median :2008   Median : 6.50   Median :39308  
 Mean   :2008   Mean   : 6.50   Mean   :39541  
 3rd Qu.:2015   3rd Qu.: 9.25   3rd Qu.:41200  
 Max.   :2021   Max.   :12.00   Max.   :56772  

In [7]:
for (i in 2: ncol(feature)){ 
    feature[,i] <- as.numeric(str_replace_all(as.character(feature[,i]),',',''))

}

In [8]:
str(feature)

'data.frame':	492 obs. of  34 variables:
 $ 년                                : int  1980 1980 1980 1980 1980 1980 1980 1980 1980 1980 ...
 $ 월                                : num  1 2 3 4 5 6 7 8 9 10 ...
 $ 곡류_수입                         : num  0 0 0 0 0 0 0 0 0 0 ...
 $ 맥류및잡곡_수입                   : num  66.6 66.6 65.8 63 62.3 ...
 $ 콩류_수입                         : num  63.6 63.6 65.7 62.7 62.7 ...
 $ 과실_수입                         : num  0 0 0 0 0 0 0 0 0 0 ...
 $ 도축육_수입                       : num  29.5 26.6 26.6 26.6 26.6 ...
 $ 가금육_수입                       : num  0 0 0 0 0 0 0 0 0 0 ...
 $ 신선어류_수입                     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ 냉동수산물_수입                   : num  29.1 29.1 29.1 29 30 ...
 $ 낙농품_수입                       : num  0 0 0 0 0 0 0 0 0 0 ...
 $ 수산물가공품_수입                 : num  0 0 0 0 0 0 0 0 0 0 ...
 $ 원당_수입                         : num  84.7 99.4 99.4 118 161.4 ...
 $ 유지_수입                         : num  59.6 59.4 58.7 50.5 50.4 ...
 $ 과실및채소가공품_수입   

In [9]:
(1996 - 1980) * 12

[1] 192

In [24]:
abc <- read.csv('../Data/rice.csv', fileEncoding = "utf-8")

In [25]:
str(abc)##c(1,2,3,4,5,17,19,27:nrow(Exam1234))

'data.frame':	252 obs. of  35 variables:
 $ 년                                : int  2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 ...
 $ 월                                : int  1 2 3 4 5 6 7 8 9 10 ...
 $ 곡류_수입                         : num  38.2 38.2 38.2 38.2 38.2 ...
 $ 맥류및잡곡_수입                   : num  46.2 46.1 45.9 46 46.5 ...
 $ 콩류_수입                         : num  49.8 50.2 50.9 50.9 52.1 ...
 $ 과실_수입                         : num  68.3 71.7 73.5 65.1 65.4 ...
 $ 도축육_수입                       : num  56.8 55.9 57 58.9 58.4 ...
 $ 가금육_수입                       : num  0 0 0 0 0 0 0 0 0 0 ...
 $ 신선어류_수입                     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ 냉동수산물_수입                   : num  69 70 71 68.4 72.6 ...
 $ 낙농품_수입                       : num  0 0 0 0 0 0 0 0 0 0 ...
 $ 수산물가공품_수입                 : num  47 45.7 45.7 45.9 45.3 ...
 $ 원당_수입                         : num  44.8 43 43 44 44 ...
 $ 유지_수입                         : num  49 47.9 48.1 48.9 50.5 ...
 $ 과실및채소가공품_수입    

In [41]:
## 타입 정해주는 if문
exex <- c()
for(i in 1:nrow(abc)){
    if(abc$rice[i]<36021){
        exex <- rbind(exex,'a')
    }else if(abc$rice[i]<39308){
        exex <- rbind(exex,'b')
    }else if(abc$rice[i]<41200){
        exex <- rbind(exex,'c')
    }else{
        exex <- rbind(exex,'d')
    }
}
head(exex)

c
c
c
c
c
c


In [42]:
table(exex)

exex
 a  b  c  d 
52 63 69 68 

In [29]:
## 정제된 전체
Exam1234 <- cbind(abc,riceType=exex)

In [38]:
## 쌀 관련만 할 경우
Exam1234 <- Exam1234[,c(1,2,3,4,5,17,19,27:ncol(Exam1234))]

In [39]:
EXAM <- function(a){
    nnet <- c()
    tree <- c()
    rf <- c()
    svm <- c()
    log <- c()
    for(i in 1:a){

        samp <- createDataPartition(Exam1234$riceType,p = 0.7,list = F)
        # 학습용
        data.tr <- Exam1234[samp,]
        # 검증용
        data.test <- Exam1234[-samp,]
        # 문제와 정답 만들기
        x <- subset(data.test,select = -riceType)
        y <- data.test$riceType
        
        model.nnet <- nnet(riceType ~ .,data = data.tr,size = 3)       # 인공 신경망
        model.tree <- ctree(riceType ~ ., data = data.tr)           # 의사 결정 나무
        model.rf <- randomForest(riceType~.,data=data.tr,ntree=100) # 랜덤 포레스트
        model.svm <- svm(riceType ~ . , data=data.tr)               # SVM
        model.log <- multinom(riceType ~ .,data=data.tr)            # 로지스틱 회귀분석
        
        #
        pred.nnet <- predict(model.nnet, x,type = 'class')
        mean.nnet <- mean(y == pred.nnet)
        nnet <- c(nnet,mean.nnet)
        #
        pred.tree <- predict(model.tree, x)
        mean.tree <- mean(y == pred.tree)
        tree <- c(tree,mean.tree)
        #
        pred.rf <- predict(model.rf, x)
        mean.rf <- mean(y == pred.rf)
        rf <- c(rf,mean.rf)
        #
        pred.svm <- predict(model.svm, x)
        mean.svm <- mean(y == pred.svm)
        svm <- c(svm,mean.svm)
        #
        pred.log <- predict(model.log, x)
        mean.log <- mean(y == pred.log)
        log <- c(log,mean.log)
        
        
    }
    cat('-----------------------------------------------------\n')
    cat('인공신경망 예측율\n',nnet,'\n')
    cat('평균 예측율 : ',mean(nnet),'\n')
    cat('의사결정나무 예측율\n',tree,'\n')
    cat('평균 예측율 : ',mean(tree),'\n')
    cat('랜덤포레스트 예측율\n',rf,'\n')
    cat('평균 예측율 : ',mean(rf),'\n')
    cat('SVM 예측율\n',svm,'\n')
    cat('평균 예측율 : ',mean(svm),'\n')
    cat('로지스틱 예측율\n',log,'\n')
    cat('평균 예측율 : ',mean(log),'\n')
    cat('-----------------------------------------------------\n')
}

In [40]:
EXAM(5)

# weights:  67
initial  value 346.011074 
final  value 247.121441 
converged
# weights:  72 (51 variable)
initial  value 248.146691 
iter  10 value 169.553201
iter  20 value 111.297143
iter  30 value 99.793348
iter  40 value 74.537492
iter  50 value 31.583064
iter  60 value 5.522501
iter  70 value 0.065434
iter  80 value 0.000227
final  value 0.000025 
converged
# weights:  67
initial  value 258.152321 
final  value 247.121441 
converged
# weights:  72 (51 variable)
initial  value 248.146691 
iter  10 value 186.976602
iter  20 value 102.615895
iter  30 value 85.281179
iter  40 value 68.880469
iter  50 value 29.674208
iter  60 value 2.396792
iter  70 value 0.011327
final  value 0.000088 
converged
# weights:  67
initial  value 259.904713 
final  value 247.121441 
converged
# weights:  72 (51 variable)
initial  value 248.146691 
iter  10 value 194.635147
iter  20 value 125.582438
iter  30 value 107.077902
iter  40 value 86.494710
iter  50 value 44.681239
iter  60 value 10.270707
iter  70